In [1]:
import pandas as pd
import regex as re
from collections import Counter
import emoji
from gensim.models.word2vec import Word2Vec
import nltk
from nltk import word_tokenize
from gensim.models import FastText
from gensim.models import KeyedVectors
#from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np
#from tqdm import tqdm
from gensim.models.callbacks import CallbackAny2Vec
#import matplotlib.pyplot as plt
import keras
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
#from keras.initializers import Constant
from keras.layers import ReLU
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense, Flatten, Embedding, Input, LSTM, Bidirectional, TimeDistributed, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.optimizers import Adam
from keras import Model
from keras.models import load_model
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import math
nltk.download('punkt')

2023-04-30 14:46:26.906654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-30 14:46:26.906748: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
[nltk_data] Downloading package punkt to /home/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df_hing_eng = pd.read_pickle('dataset/hinglish_english_tweet_ids_data.pkl')
#print(df_hing_eng.columns)
#print(df_hing_eng[['tweet_text']])
#df_hing_eng = df_hing_eng['tweet_text']
df_hing_eng = df_hing_eng[['tweet_text']].copy()
print(len(df_hing_eng))
print(df_hing_eng.columns)

422415
Index(['tweet_text'], dtype='object')


In [3]:
#finds the duplicate rows in the unlabeled Hinglish-English data

duplicate_1 = df_hing_eng[df_hing_eng.duplicated(keep='first')]
print(len(duplicate_1))
print(duplicate_1)
#print(type(duplicate_1))

17695
                                               tweet_text
368     Chitti ai hai ai hai chitthi ai hai = I have m...
372     Chitti ai hai ai hai chitthi ai hai = I have m...
393                    Kim Kardashian is virgin. #sarcasm
394                    Kim Kardashian is virgin. #sarcasm
423                    Kim Kardashian is virgin. #sarcasm
...                                                   ...
422093                                                 hi
422100                                   is going to bed.
422121                                             nothin
422141                                           wakes up
422251                                      in the office

[17695 rows x 1 columns]


In [4]:
#removes duplicates

df_hing_eng = df_hing_eng.drop_duplicates(keep='first')
print(len(df_hing_eng))

duplicate_2 = df_hing_eng[df_hing_eng.duplicated(keep='first')]
print(len(duplicate_2))
print(df_hing_eng.columns)

404720
0
Index(['tweet_text'], dtype='object')


In [5]:
df_labeled = pd.read_pickle('dataset/data_soumyadeep.pkl')
df_labeled = df_labeled[['tweet_text', 'is_sarcastic']].copy()
print(len(df_labeled))
print(df_labeled.columns)

94788
Index(['tweet_text', 'is_sarcastic'], dtype='object')


In [6]:
#finds the duplicate rows in the labeled data

duplicate_3 = df_labeled[df_labeled.duplicated(subset=['tweet_text'], keep='first')]
print(len(duplicate_3))
print(duplicate_3)

2895
                                              tweet_text  is_sarcastic
354    RT Apna vyauhar hoga aise jal mein kamal rehat...             0
630    @fittaymunhapka n osky baad amMi ny kha k kuch...             0
686    RT Teen behan ke baad jo aata putra woh trekha...             0
809    Ek aadmi pareshan ho kar Ae bhagwan aisi zinda...             1
844    RT Teen behan ke baad jo aata putra woh trekha...             0
...                                                  ...           ...
94716  RT Teen behan ke baad jo aata putra woh trekha...             0
94721  Roz badh'taa huun jahaa'n se aage // Phir wahi...             0
94747  RT Chhoota vaibhav schooli shiksha shooro ho g...             0
94756                                kya hoga is desh ka             0
94763  @rjauhari lol  unka bhi mast hoga..koi recessi...             0

[2895 rows x 2 columns]


In [7]:
#removes duplicates from labeled data

df_labeled = df_labeled.drop_duplicates(subset=['tweet_text'], keep='first')
print(len(df_labeled))

duplicate_4 = df_labeled[df_labeled.duplicated(keep='first')]
print(len(duplicate_4))
print(df_labeled.columns)

91893
0
Index(['tweet_text', 'is_sarcastic'], dtype='object')


In [8]:
#finds the common rows between unlabeled Hinglish English embedding data and labeled data

common_rows = pd.merge(df_hing_eng, df_labeled, how = 'inner', on=['tweet_text'])
print(common_rows)
len(common_rows)
#print(type(common_rows))

                                              tweet_text  is_sarcastic
0      @ajuonline #sarcasm Haal chaal thik thak hai-s...             1
1               @I_Am_Hollywood - Charlie Haas? #sarcasm             1
2                       Chetan Bhagat #epicmale #sarcasm             1
3      @Bo_jasim86 LOOL 7atheer affaa 3allaik rgoobti...             1
4      @FENUSF tees2al 3anek el 3afiaa 7abeeb galbe ,...             1
...                                                  ...           ...
54025   BHAI NEEND HEE NI ARAHI AKHIR KIA MAZAK HAI??)??             1
54026  - Neend Toh Bachpan Mein Aaya Karti Thi..!! - ...             1
54027  aik larki late night neend se jagi aur.. fb pe...             1
54028  @FarahKhanAli really funny.. lagta hai.... sub...             1
54029  @Rudeicious ab tum meri neend ka aise Mazak ur...             1

[54030 rows x 2 columns]


54030

In [9]:
#removes all the common rows from unlabeled data

#cond = df1['Email'].isin(df2['Email'])
#df1.drop(df1[cond].index, inplace = True)

common = df_hing_eng['tweet_text'].isin(df_labeled['tweet_text'])
df_hing_eng.drop(df_hing_eng[common].index, inplace=True)
print(len(df_hing_eng))
print(df_hing_eng.columns)

common_rows2 = pd.merge(df_hing_eng, df_labeled, how = 'inner', on=['tweet_text'])
#print(common_rows2)
print(len(common_rows2))

350690
Index(['tweet_text'], dtype='object')
0


In [10]:
#removes all rows from the Dataframe containing Devanagari script (any non-English word) 

df_hing_eng = df_hing_eng[df_hing_eng['tweet_text'].map(lambda x: x.isascii())]
print(len(df_hing_eng))

318723


In [11]:
#removes mentions, '#', emojis, punctuations, special chars, URLs, words containing numbers, spaces from beginning and end of tweets

def data_preprocessing(source):
    
    source = source.str.lower() #lower case chars
    source = source.str.replace("\w*\d\w*","") #removes all words containing numbers
    #source = source.str.replace("#", "")
    #source = source.str.replace("@", "")
    #source = source.str.replace("@[A-Za-z0-9_]+","")
    source = source.str.replace("[@A-Za-z0-9_]*@[@A-Za-z0-9_]*","")
    #source = source.str.replace("#[A-Za-z0-9_]+","")
    source = source.str.replace("[#A-Za-z0-9_]*#[#A-Za-z0-9_]*","")
    source = source.str.replace("\s+|[!$%&()*+, -./:;<=>?\^_`{|}~]\s*"," ")
    source = source.str.replace("http\S+","")
    
    return source

df_hing_eng['tweet_text'] = data_preprocessing(df_hing_eng['tweet_text'])
df_hing_eng['tweet_text'] = df_hing_eng['tweet_text'].apply(lambda s: emoji.replace_emoji(s,""))
df_hing_eng['tweet_text'] = df_hing_eng['tweet_text'].str.strip()
df_hing_eng.head()

/tmp/ipykernel_6642/2924867879.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("\w*\d\w*","") #removes all words containing numbers
/tmp/ipykernel_6642/2924867879.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("[@A-Za-z0-9_]*@[@A-Za-z0-9_]*","")
/tmp/ipykernel_6642/2924867879.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("[#A-Za-z0-9_]*#[#A-Za-z0-9_]*","")
/tmp/ipykernel_6642/2924867879.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("\s+|[!$%&()*+, -./:;<=>?\^_`{|}~]\s*"," ")
/tmp/ipykernel_6642/2924867879.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("http\S+","")


,tweet_text
167,dary emratiba
194,yograj singh dhoni advised dd management to r...
200,seedhe log hai ma beta ab crore ka ghotala ho...
309,ab ham khade ho gaye hain ab ham aur bhi dar...
470,make up hayoona


In [12]:
#finds the unique words in the unlabeled data

unique_words = set()
df_hing_eng['tweet_text'].str.lower().str.split().apply(unique_words.update)
unique_words = list(unique_words)
#print(unique_words)
#print('---------------')
print(len(unique_words))

197446


In [13]:
#generates tokenized data 

#nltk.download('punkt')
df_hing_eng['tweet_text'] = df_hing_eng['tweet_text'].apply(nltk.word_tokenize)
df_hing_eng.head(10)

,tweet_text
167,"[dary, emratiba]"
194,"[yograj, singh, dhoni, advised, dd, management..."
200,"[seedhe, log, hai, ma, beta, ab, crore, ka, gh..."
309,"[ab, ham, khade, ho, gaye, hain, ab, ham, aur,..."
470,"[make, up, hayoona]"
571,"[cut, marna, koi, umar, se, seekhay, nahi, see..."
804,"[woh, jo, kar, rahe, haen, woh, desh, sewa, ha..."
905,"[u, r, aapne, chaata, nahi, ki, tarah, pehle, ..."
1077,"[bhai, tu, ek, vaio, mini, laptop, le, le]"
1277,"[aanaayi, janichirunengil, kashtapettu, cash, ..."


In [14]:
#removes all the rows with empty lists

df_hing_eng=df_hing_eng[df_hing_eng['tweet_text'].map(lambda d: len(d)) > 0]
df_hing_eng.head(10)

,tweet_text
167,"[dary, emratiba]"
194,"[yograj, singh, dhoni, advised, dd, management..."
200,"[seedhe, log, hai, ma, beta, ab, crore, ka, gh..."
309,"[ab, ham, khade, ho, gaye, hain, ab, ham, aur,..."
470,"[make, up, hayoona]"
571,"[cut, marna, koi, umar, se, seekhay, nahi, see..."
804,"[woh, jo, kar, rahe, haen, woh, desh, sewa, ha..."
905,"[u, r, aapne, chaata, nahi, ki, tarah, pehle, ..."
1077,"[bhai, tu, ek, vaio, mini, laptop, le, le]"
1277,"[aanaayi, janichirunengil, kashtapettu, cash, ..."


In [15]:
#model = Word2Vec(df_hing_eng['tweet_text'], vector_size=300, window=10, hs=0, negative = 1)
model = FastText(df_hing_eng['tweet_text'], vector_size=300, window=10, hs=0, negative = 1)
embedding_size = model.wv.vectors.shape[1]
print("embedding size--->", embedding_size)
vocab = model.wv.index_to_key

embedding size---> 300


In [16]:
#saves the FastText model

model.save("hing_eng_fasttext.model")
print("Model saved!!!")

Model saved!!!


In [17]:
embeddings_index = FastText.load("hing_eng_fasttext.model")
print(embeddings_index)

FastText(vocab=39876, vector_size=300, alpha=0.025)


In [18]:
print(df_labeled.head())
print(len(df_labeled))

                                          tweet_text  is_sarcastic
0  LOL !  BC badaa gaandu hai to ! so jaa lodu ! ...             1
1  @OFFICIAL_FC_PAK apni sakh khud sambhalen aur ...             0
2  Ek aadmi pareshan ho kar Ae bhagwan aisi zinda...             1
3  Lol RT @ThePrachi: @shrikhande Who can sing a ...             1
4  @idrinkblueblood tu ol's kar rahi ho kya? Kya ...             0
91893


In [19]:
#removes mentions, '#', emojis, punctuations, special chars, URLs, words containing numbers, spaces from beginning and end of tweets

def data_preprocessing(source):
    
    source = source.str.lower() #lower case chars
    source = source.str.replace("\w*\d\w*","") #removes all words containing numbers
    #source = source.str.replace("#", "")
    #source = source.str.replace("@", "")
    #source = source.str.replace("@[A-Za-z0-9_]+","")
    source = source.str.replace("[@A-Za-z0-9_]*@[@A-Za-z0-9_]*","")
    #source = source.str.replace("#[A-Za-z0-9_]+","")
    source = source.str.replace("[#A-Za-z0-9_]*#[#A-Za-z0-9_]*","")
    source = source.str.replace("\s+|[!$%&()*+, -./:;<=>?\^_`{|}~]\s*"," ")
    source = source.str.replace("http\S+","")
    
    return source

df_labeled['tweet_text'] = data_preprocessing(df_labeled['tweet_text'])
df_labeled['tweet_text'] = df_labeled['tweet_text'].apply(lambda s: emoji.replace_emoji(s,""))
df_labeled['tweet_text'] = df_labeled['tweet_text'].str.strip()
df_labeled[['tweet_text']].head()

/tmp/ipykernel_6642/3018622983.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("\w*\d\w*","") #removes all words containing numbers
/tmp/ipykernel_6642/3018622983.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("[@A-Za-z0-9_]*@[@A-Za-z0-9_]*","")
/tmp/ipykernel_6642/3018622983.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("[#A-Za-z0-9_]*#[#A-Za-z0-9_]*","")
/tmp/ipykernel_6642/3018622983.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("\s+|[!$%&()*+, -./:;<=>?\^_`{|}~]\s*"," ")
/tmp/ipykernel_6642/3018622983.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  source = source.str.replace("http\S+","")


,tweet_text
0,lol bc badaa gaandu hai to so jaa lodu tere...
1,apni sakh khud sambhalen aur kisi ka saath na ...
2,ek aadmi pareshan ho kar ae bhagwan aisi zinda...
3,lol rt who can sing a group song alone raavan...
4,tu ol s kar rahi ho kya kya hoga tere kaam sir...


In [20]:
df_labeled.head()

,tweet_text,is_sarcastic
0,lol bc badaa gaandu hai to so jaa lodu tere...,1
1,apni sakh khud sambhalen aur kisi ka saath na ...,0
2,ek aadmi pareshan ho kar ae bhagwan aisi zinda...,1
3,lol rt who can sing a group song alone raavan...,1
4,tu ol s kar rahi ho kya kya hoga tere kaam sir...,0


In [22]:
#finds the unique words in the labeled data

unique_words = set()
df_labeled['tweet_text'].str.lower().str.split().apply(unique_words.update)
unique_words = list(unique_words)
#print(unique_words)
#print('---------------')
print(len(unique_words))

93958


In [23]:
#generates tokenized data 

#nltk.download('punkt')
df_labeled['tweet_text'] = df_labeled['tweet_text'].apply(nltk.word_tokenize)
df_labeled.head()

,tweet_text,is_sarcastic
0,"[lol, bc, badaa, gaandu, hai, to, so, jaa, lod...",1
1,"[apni, sakh, khud, sambhalen, aur, kisi, ka, s...",0
2,"[ek, aadmi, pareshan, ho, kar, ae, bhagwan, ai...",1
3,"[lol, rt, who, can, sing, a, group, song, alon...",1
4,"[tu, ol, s, kar, rahi, ho, kya, kya, hoga, ter...",0


In [24]:
#removes all the rows with empty lists

print(len(df_labeled))
df_labeled = df_labeled[df_labeled['tweet_text'].map(lambda d: len(d)) > 0]
print(df_labeled.head(10))
print(len(df_labeled))

91893
                                          tweet_text  is_sarcastic
0  [lol, bc, badaa, gaandu, hai, to, so, jaa, lod...             1
1  [apni, sakh, khud, sambhalen, aur, kisi, ka, s...             0
2  [ek, aadmi, pareshan, ho, kar, ae, bhagwan, ai...             1
3  [lol, rt, who, can, sing, a, group, song, alon...             1
4  [tu, ol, s, kar, rahi, ho, kya, kya, hoga, ter...             0
5      [yaar, ye, log, tu, serious, ho, gaye, t, co]             0
6  [abey, tumhare, paas, kaam, nahi, hai, jo, twi...             0
7  [ani, tauko, dukhyo, vanera, jaane, ta, ho, ni...             0
8  [rt, agar, tumne, rt, nahi, kiya, to, mujhse, ...             0
9  [dil, ki, dhadkan, sooni, sooni, saansain, bhi...             0
91892


In [25]:
df_labeled.head()

,tweet_text,is_sarcastic
0,"[lol, bc, badaa, gaandu, hai, to, so, jaa, lod...",1
1,"[apni, sakh, khud, sambhalen, aur, kisi, ka, s...",0
2,"[ek, aadmi, pareshan, ho, kar, ae, bhagwan, ai...",1
3,"[lol, rt, who, can, sing, a, group, song, alon...",1
4,"[tu, ol, s, kar, rahi, ho, kya, kya, hoga, ter...",0


In [26]:
df_hing_eng.head()

,tweet_text
167,"[dary, emratiba]"
194,"[yograj, singh, dhoni, advised, dd, management..."
200,"[seedhe, log, hai, ma, beta, ab, crore, ka, gh..."
309,"[ab, ham, khade, ho, gaye, hain, ab, ham, aur,..."
470,"[make, up, hayoona]"


In [27]:
#train test split of labeled data

check_nan = df_labeled['is_sarcastic'].isnull().values.any()
print(check_nan)
check_na = df_labeled['is_sarcastic'].isna().values.any()
print(check_na)
check_nan_ = df_labeled['tweet_text'].isnull().values.any()
print(check_nan_)
check_na_ = df_labeled['tweet_text'].isna().values.any()
print(check_na_)

x_train,x_test,y_train,y_test=train_test_split(df_labeled['tweet_text'], df_labeled['is_sarcastic'], test_size=0.30, random_state=42)
#x_train,x_test,y_train,y_test=train_test_split(padded_tweets, sarcasm_labels, test_size=0.20, random_state=42)
print("x_train_shape--->", x_train.shape)
print("y_train_shape--->", y_train.shape)
print("x_test_shape--->", x_test.shape)
print("y_test_shape--->", y_test.shape)
print(type(x_train))
print("contents of first label--->", y_train[0])

False
False
False
False
x_train_shape---> (64324,)
y_train_shape---> (64324,)
x_test_shape---> (27568,)
y_test_shape---> (27568,)
<class 'pandas.core.series.Series'>
contents of first label---> 1


In [28]:
print(x_train)

39384    [kisi, siyaasat, daan, ki, taraah, tumhen, haa...
70407    [agar, bharat, ko, hai, mahaan, banna, toh, bh...
12711    [atal, bihari, vajpayee, and, nawaz, sharif, s...
20467    [bara, shok, tha, unhain, mera, ayeshana, daik...
15898    [wo, aap, ne, sunna, hee, hoga, goray, kaam, k...
                               ...                        
6316     [oye, yaar, tera, dimag, kharab, hai, kya, abh...
56186    [mulke, halate, jung, mein, hai, mujhe, koi, s...
79025    [mastar, bandya, mashi, aani, daasa, madhe, ka...
865      [ek, kunwari, larki, pehli, baar, ek, larke, s...
16017    [always, joking, jo, chij, kbhi, sudhar, nhi, ...
Name: tweet_text, Length: 64324, dtype: object


In [29]:
print(y_train.value_counts())
print('--------------------------')
print(y_test.value_counts())

0    32482
1    31842
Name: is_sarcastic, dtype: int64
--------------------------
0    13962
1    13606
Name: is_sarcastic, dtype: int64


In [30]:
length = []
for row in x_train:
    #print(row)
    #print(len(row))
    #count = count+1
    #if len(row)>maxi:
    #print(len(row))
    length.append(len(row))
        
#print(length)
print("Max length--->", max(length))
print("Mean length--->", np.mean(length))
print("Standard Deviation of length--->", np.std(length))

#maxi = max(length)
maxi = np.round(np.mean(length) + np.std(length)).astype(int)
print("Maximum length of the sequence--->", maxi)

Max length---> 84
Mean length---> 16.626049375038864
Standard Deviation of length---> 9.42481679865071
Maximum length of the sequence---> 26


In [31]:
total_docs=[]
total_docs.extend(x_train)
total_docs.extend(x_test)
print(total_docs[0:5])

[['kisi', 'siyaasat', 'daan', 'ki', 'taraah', 'tumhen', 'haasil', 'karne', 'k', 'liye', 'roz', 'naye', 'jalse', 'karta', 'hoon', 'apne', 'dil', 'men'], ['agar', 'bharat', 'ko', 'hai', 'mahaan', 'banna', 'toh', 'bhrasht', 'netaao', 'ko', 'hoga', 'hatna', 'aur', 'bhrashtachaar', 'ko', 'hoga', 'mitaana', 'yeh', 'kisi', 'ek', 'seh', 'na', 'hoga', 'pureh', 'jansamuday', 'ko', 'hoga', 'saath', 'nibhaana', 'happy', 'independence', 'day'], ['atal', 'bihari', 'vajpayee', 'and', 'nawaz', 'sharif', 'share', 'the', 'same', 'date', 'of', 'birth'], ['bara', 'shok', 'tha', 'unhain', 'mera', 'ayeshana', 'daiknay', 'ka', 'naddan', 'jab', 'daiki', 'meri', 'garabi', 'tu', 'rasta', 'badal', 'liya'], ['wo', 'aap', 'ne', 'sunna', 'hee', 'hoga', 'goray', 'kaam', 'ko', 'dimaag', 'mei', 'rakhtay', 'hain', 'aur', 'hehe', 'magar', 'pakistani', 'uss', 'ko', 'dimag', 'mei', 'aur', 'kaam', 'ko']]


In [32]:
print("tokenizing input data...")
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(total_docs)
#tokenizer.fit_on_texts(x_train)
word_seq_train = tokenizer.texts_to_sequences(x_train)
word_seq_test = tokenizer.texts_to_sequences(x_test)
word_index = tokenizer.word_index

tokenizing input data...


In [33]:
print(len(word_seq_train))
print(word_seq_train[0])
print(len(word_seq_test))
print(word_seq_test[0])
print("dictionary size: ", len(word_index))

64324
[58, 17091, 5027, 2, 25830, 1972, 4437, 122, 11, 42, 391, 1217, 9317, 149, 163, 87, 13, 380]
27568
[695, 4, 113, 623, 14, 172, 16, 76773, 5277, 7, 113, 623]
dictionary size:  93854


In [34]:
#pads to the length of the longest sequence

word_seq_train = pad_sequences(word_seq_train, maxlen=maxi, padding='post')
word_seq_test = pad_sequences(word_seq_test, maxlen=maxi, padding='post')
print(word_seq_train[0])
print(word_seq_train.shape)
print(word_seq_test.shape)

[   58 17091  5027     2 25830  1972  4437   122    11    42   391  1217
  9317   149   163    87    13   380     0     0     0     0     0     0
     0     0]
(64324, 26)
(27568, 26)


In [35]:
#builds the embedding matrix

print('preparing embedding matrix...')
embed_dim = embedding_size
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index))
embedding_matrix = np.random.rand(nb_words+1, embed_dim)

for word, i in word_index.items():
    if i >= nb_words:
        continue
    #print(word)

    if embeddings_index.wv.__contains__(word): 

        embedding_vector = embeddings_index.wv[word]
    
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
#print(embedding_matrix)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print(embedding_matrix.shape)
print(len(embedding_matrix[34]))

preparing embedding matrix...
number of null word embeddings: 0
(93855, 300)
300


In [36]:
print(words_not_found)

[]


In [37]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

In [38]:
#Attention BiLSTM

from keras import backend as K
#from keras.engine.topology import Layer
from tensorflow.keras.layers import Layer
from keras import initializers, regularizers, constraints


class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight( shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]
        #print(self.name)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [39]:
lstm_out1 = 150
#bilstm_out1 = 96
#lstm_out2 = 32
#dropout = 0.2
#recurrent_dropout = 0.2

inp = Input(shape=(maxi, ))
x = Embedding(nb_words+1, embed_dim, weights=[embedding_matrix], trainable=False)(inp)
x = Bidirectional(LSTM(lstm_out1, return_sequences=True, dropout=0.20, recurrent_dropout=0.20))(x)
x = Attention(maxi)(x)
x = Dense(256, activation="relu")(x)
#x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

2023-04-30 15:16:23.947246: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-04-30 15:16:23.972480: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-30 15:16:24.019975: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (user-PC): /proc/driver/nvidia/version does not exist
2023-04-30 15:16:24.280222: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-30 15:16:29.343299: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 112626000 exceeds 1

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 26)]              0         
                                                                 
 embedding (Embedding)       (None, 26, 300)           28156500  
                                                                 
 bidirectional (Bidirectiona  (None, 26, 300)          541200    
 l)                                                              
                                                                 
 attention (Attention)       (None, 300)               326       
                                                                 
 dense (Dense)               (None, 256)               77056     
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                             

/home/user/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [40]:

#trains attention BiLSTM model 

model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
#model.save('att_bilstm.h5') 

Epoch 1/20
905/905 [==============================] - 285s 303ms/step - loss: 0.2548 - accuracy: 0.9118 - val_loss: 0.1667 - val_accuracy: 0.9532
Epoch 2/20
905/905 [==============================] - 271s 299ms/step - loss: 0.1597 - accuracy: 0.9517 - val_loss: 0.1596 - val_accuracy: 0.9520
Epoch 3/20
905/905 [==============================] - 271s 300ms/step - loss: 0.1451 - accuracy: 0.9546 - val_loss: 0.1497 - val_accuracy: 0.9541
Epoch 4/20
905/905 [==============================] - 269s 297ms/step - loss: 0.1315 - accuracy: 0.9570 - val_loss: 0.1499 - val_accuracy: 0.9534
Epoch 5/20
905/905 [==============================] - 271s 300ms/step - loss: 0.1165 - accuracy: 0.9608 - val_loss: 0.1523 - val_accuracy: 0.9543
Epoch 6/20
905/905 [==============================] - 303s 335ms/step - loss: 0.1008 - accuracy: 0.9656 - val_loss: 0.1544 - val_accuracy: 0.9568
Epoch 7/20
905/905 [==============================] - 307s 339ms/step - loss: 0.0875 - accuracy: 0.9696 - val_loss: 0.1797 -

In [41]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

862/862 [==============================] - 52s 60ms/step
Accuracy is---> 95.09939059779454


In [42]:
#this one--->LSTM

lstm_out1 = 150
'''bilstm_out1 = 96
lstm_out2 = 32
dropout = 0.2
recurrent_dropout = 0.2'''

model = Sequential()
model.add(Embedding(nb_words+1, embed_dim, weights=[embedding_matrix], input_length=maxi, trainable=False))
model.add(LSTM(lstm_out1, dropout=0.2, recurrent_dropout=0.2))

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='binary_crossentropy',
                  optimizer=adam,
                  metrics=['accuracy'])

model.summary()

2023-04-30 19:42:30.398971: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 112626000 exceeds 10% of free system memory.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 26, 300)           28156500  
                                                                 
 lstm_1 (LSTM)               (None, 150)               270600    
                                                                 
 dense_2 (Dense)             (None, 64)                9664      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 28,436,829
Trainable params: 280,329
Non-trainable params: 28,156,500
_________________________________________________________________


/home/user/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [43]:
#trains the LSTM model

model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
model.save('lstm.h5') 

Epoch 1/20
905/905 [==============================] - 186s 197ms/step - loss: 0.3442 - accuracy: 0.8596 - val_loss: 0.2442 - val_accuracy: 0.9179
Epoch 2/20
905/905 [==============================] - 137s 152ms/step - loss: 0.2036 - accuracy: 0.9361 - val_loss: 0.1796 - val_accuracy: 0.9484
Epoch 3/20
905/905 [==============================] - 131s 145ms/step - loss: 0.1705 - accuracy: 0.9484 - val_loss: 0.1677 - val_accuracy: 0.9510
Epoch 4/20
905/905 [==============================] - 129s 143ms/step - loss: 0.1570 - accuracy: 0.9515 - val_loss: 0.1605 - val_accuracy: 0.9531
Epoch 5/20
905/905 [==============================] - 145s 161ms/step - loss: 0.1454 - accuracy: 0.9540 - val_loss: 0.1616 - val_accuracy: 0.9517
Epoch 6/20
905/905 [==============================] - 191s 211ms/step - loss: 0.1390 - accuracy: 0.9556 - val_loss: 0.1618 - val_accuracy: 0.9527
Epoch 7/20
905/905 [==============================] - 137s 152ms/step - loss: 0.1293 - accuracy: 0.9578 - val_loss: 0.1651 -

In [44]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

862/862 [==============================] - 26s 30ms/step
Accuracy is---> 95.14654672083576


In [45]:
# BiLSTM

lstm_out1 = 150
bilstm_out1 = 96
lstm_out2 = 32
dropout = 0.2
recurrent_dropout = 0.2

# main model
inputt = Input(shape=(maxi,))
model = Embedding(nb_words+1, embed_dim, weights=[embedding_matrix],input_length=maxi,trainable = False)(inputt)
model =  Bidirectional (LSTM (lstm_out1, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat')(model)
model = TimeDistributed(Dense(10,activation='relu'))(model)
model = Flatten()(model)
model = Dense(64,activation='relu')(model)
output = Dense(1,activation='sigmoid')(model)
model = Model(inputt,output)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 26)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 26, 300)           28156500  
                                                                 
 bidirectional_1 (Bidirectio  (None, 26, 300)          541200    
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 26, 10)           3010      
 ibuted)                                                         
                                                                 
 flatten (Flatten)           (None, 260)               0         
                                                                 
 dense_5 (Dense)             (None, 64)                1670

/home/user/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [46]:
model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
model.save('bilstm.h5') 

Epoch 1/20
905/905 [==============================] - 333s 289ms/step - loss: 0.3091 - accuracy: 0.8817 - val_loss: 0.1970 - val_accuracy: 0.9422
Epoch 2/20
905/905 [==============================] - 391s 432ms/step - loss: 0.1715 - accuracy: 0.9486 - val_loss: 0.1616 - val_accuracy: 0.9523
Epoch 3/20
905/905 [==============================] - 318s 352ms/step - loss: 0.1437 - accuracy: 0.9547 - val_loss: 0.1470 - val_accuracy: 0.9554
Epoch 4/20
905/905 [==============================] - 346s 382ms/step - loss: 0.1243 - accuracy: 0.9591 - val_loss: 0.1440 - val_accuracy: 0.9543
Epoch 5/20
905/905 [==============================] - 304s 336ms/step - loss: 0.1089 - accuracy: 0.9636 - val_loss: 0.1535 - val_accuracy: 0.9546
Epoch 6/20
905/905 [==============================] - 334s 369ms/step - loss: 0.0947 - accuracy: 0.9678 - val_loss: 0.1477 - val_accuracy: 0.9569
Epoch 7/20
905/905 [==============================] - 343s 379ms/step - loss: 0.0817 - accuracy: 0.9716 - val_loss: 0.1601 -

In [47]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

862/862 [==============================] - 63s 71ms/step
Accuracy is---> 95.3786999419617


In [48]:
#CNN1

#model parameters

num_kernels=200
stride=1
embed_dim = 300
weight_decay = 1e-4

#parallel layers 
kernel_size=7
kernel_size_p1=3
kernel_size_p2=6
kernel_size_p3=9
kernel_size_p4=12

#CNN architecture
print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words+1, embed_dim, weights=[embedding_matrix], input_length=maxi, trainable=False))
model.add(Conv1D(num_kernels, kernel_size, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_kernels, kernel_size, activation='relu', padding='same'))
#model.add(GlobalMaxPooling1D())
model.add(MaxPooling1D(2))
#model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
#model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)
model.add(Dense(1, activation='sigmoid')) 

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

training CNN ...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 26, 300)           28156500  
                                                                 
 conv1d (Conv1D)             (None, 26, 200)           420200    
                                                                 
 max_pooling1d (MaxPooling1D  (None, 13, 200)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 200)           280200    
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 6, 200)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d (Globa  (None, 

/home/user/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [49]:
model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
model.save('cnn1.h5') 

Epoch 1/20
905/905 [==============================] - 162s 177ms/step - loss: 0.4376 - accuracy: 0.8060 - val_loss: 0.3060 - val_accuracy: 0.8899
Epoch 2/20
905/905 [==============================] - 154s 170ms/step - loss: 0.3019 - accuracy: 0.8935 - val_loss: 0.2772 - val_accuracy: 0.9060
Epoch 3/20
905/905 [==============================] - 146s 161ms/step - loss: 0.2747 - accuracy: 0.9057 - val_loss: 0.2799 - val_accuracy: 0.9018
Epoch 4/20
905/905 [==============================] - 108s 120ms/step - loss: 0.2538 - accuracy: 0.9124 - val_loss: 0.2696 - val_accuracy: 0.9035
Epoch 5/20
905/905 [==============================] - 111s 123ms/step - loss: 0.2387 - accuracy: 0.9168 - val_loss: 0.2749 - val_accuracy: 0.8997
Epoch 6/20
905/905 [==============================] - 112s 124ms/step - loss: 0.2230 - accuracy: 0.9229 - val_loss: 0.2644 - val_accuracy: 0.9081
Epoch 7/20
905/905 [==============================] - 112s 124ms/step - loss: 0.2055 - accuracy: 0.9284 - val_loss: 0.2586 -

In [50]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

862/862 [==============================] - 19s 21ms/step
Accuracy is---> 91.52278003482299


In [51]:
#CNN2

#num_classes = len(label_names)
num_classes = 2

inputt = Input(shape=(maxi,))
model = Embedding(nb_words+1, embed_dim, weights=[embedding_matrix],input_length=maxi,trainable = False)(inputt)
tower_1 = Conv1D(num_kernels, kernel_size_p1, padding='same', activation='relu', strides=stride)(model)
tower_2 = Conv1D(num_kernels, kernel_size_p2, padding='same', activation='relu', strides=stride)(model)
tower_3 = Conv1D(num_kernels, kernel_size_p3, padding='same', activation='relu', strides=stride)(model)
tower_4 = Conv1D(num_kernels, kernel_size_p4, padding='same', activation='relu', strides=stride)(model)
model = keras.layers.concatenate([tower_1, tower_2, tower_3, tower_4], axis=1)
model = GlobalMaxPooling1D()(model)
model = Dropout(0.5)(model)
#model = Flatten()(model)
model = Dense(128, activation='relu')(model)
model = Dense(64, activation='relu')(model)
#output = Dense(num_classes, activation='sigmoid')(model)
output = Dense(1, activation='sigmoid')(model)
model = Model(inputt, output)

#adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 26)]         0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, 26, 300)      28156500    ['input_3[0][0]']                
                                                                                                  
 conv1d_2 (Conv1D)              (None, 26, 200)      180200      ['embedding_4[0][0]']            
                                                                                                  
 conv1d_3 (Conv1D)              (None, 26, 200)      360200      ['embedding_4[0][0]']            
                                                                                            

In [52]:
model.fit(word_seq_train, y_train, batch_size=64, epochs=20, validation_split=0.1, shuffle=True, callbacks = callbacks_list)
model.save('cnn2.h5') 

Epoch 1/20
905/905 [==============================] - 262s 287ms/step - loss: 0.5309 - accuracy: 0.7439 - val_loss: 0.3663 - val_accuracy: 0.8540
Epoch 2/20
905/905 [==============================] - 336s 371ms/step - loss: 0.3650 - accuracy: 0.8576 - val_loss: 0.3019 - val_accuracy: 0.8952
Epoch 3/20
905/905 [==============================] - 297s 328ms/step - loss: 0.3196 - accuracy: 0.8814 - val_loss: 0.2917 - val_accuracy: 0.8921
Epoch 4/20
905/905 [==============================] - 254s 280ms/step - loss: 0.2995 - accuracy: 0.8900 - val_loss: 0.2741 - val_accuracy: 0.9066
Epoch 5/20
905/905 [==============================] - 297s 328ms/step - loss: 0.2826 - accuracy: 0.8976 - val_loss: 0.2662 - val_accuracy: 0.9081
Epoch 6/20
905/905 [==============================] - 264s 291ms/step - loss: 0.2704 - accuracy: 0.9040 - val_loss: 0.2564 - val_accuracy: 0.9102
Epoch 7/20
905/905 [==============================] - 226s 249ms/step - loss: 0.2589 - accuracy: 0.9074 - val_loss: 0.2548 -

In [53]:
#model = load_model('att_bilstm.h5')
y_pred_proba = model.predict(word_seq_test)
#print(y_pred_proba)
#print('---------------')

y_pred_class = np.round(y_pred_proba)
y_pred_class = y_pred_class.squeeze() #makes y_pred_class 1 dim
'''print(y_pred_class)
print(type(y_pred_class))
print(len(y_pred_class))
print(len(y_test))
print('---------------')'''

acc = accuracy_score(y_test, y_pred_class) * 100
print("Accuracy is--->", acc)

862/862 [==============================] - 54s 62ms/step
Accuracy is---> 92.04875217643644
